# Using pandas Series

A Series is a 1D object containing an array of any NumPy data type. The key difference is that a Series object has an associated array of data labels, called the **index**.

If we do not specify the index, numerical indexes are used like for NumPy arrays.

In [1]:
import numpy as np
import pandas as pd

# Create the array
array = np.random.randint(1, 100, 10)
print array
print type(array)


# Create Series from array
series = pd.Series(array)
print series
print type(series)
print type(series.values)
print type(series.index)
# Observe that the printing is different!

[ 9 28 70  7 31 50 90 32 71 77]
<type 'numpy.ndarray'>
0     9
1    28
2    70
3     7
4    31
5    50
6    90
7    32
8    71
9    77
dtype: int64
<class 'pandas.core.series.Series'>
<type 'numpy.ndarray'>
<class 'pandas.indexes.range.RangeIndex'>


## Indexes in series

The index is a different array, and it can be changed! (or you can provide it when you create the series)

In [2]:
print(series.index)
print type(series.index)

# Change the index:
series.index=[list("abcdefghij")]
print series
print type(series.index)

RangeIndex(start=0, stop=10, step=1)
<class 'pandas.indexes.range.RangeIndex'>
a     9
b    28
c    70
d     7
e    31
f    50
g    90
h    32
i    71
j    77
dtype: int64
<class 'pandas.indexes.base.Index'>


**Reflection:** then a Series can be thought of as a **ordered** dictionary with fixed keys. Actually, it behaves as a Python dict in some situations, and you can create series from Python dicts.

In [3]:
print "a" in series
print "x" in series
print series["c"]

True
False
70


You can select using values in the index

In [4]:
series[["a","c","f"]]

a     9
c    70
f    50
dtype: int64

## Series use indexes in a smart way

Let's create two series with different indexes.

In [5]:
import pandas as pd
import numpy as np
sales1Q = pd.Series(np.random.randint(20,100, 4), 
                    index=["Europe", "USA", "LATAM", "Asia"])
sales1Q.index.name = "regions"
print sales1Q

print "--------"

sales2Q = pd.Series(np.random.randint(20,100, 3), 
                    index=["Europe", "USA", "Asia"])
print sales2Q

regions
Europe    48
USA       84
LATAM     54
Asia      84
dtype: int64
--------
Europe    60
USA       67
Asia      59
dtype: int64


Now let's try to combine them.

In [6]:
sales1Q + sales2Q

Asia      143.0
Europe    108.0
LATAM       NaN
USA       151.0
dtype: float64

It took the labels at the indexes to add the values.
But the problem is that "LATAM" was not in the second index, and then it was assumed it had no value, resulting in a NaN. We must care of missing values and replace them accordingly or remove the corresponding data depending on our data.

In [7]:
sales1Q.add(sales2Q, fill_value=0)

Asia      143.0
Europe    108.0
LATAM      54.0
USA       151.0
dtype: float64

## Manipulating Series

You can use dates to index a Series

In [8]:
dates = pd.date_range('20130125',periods=10)

values = pd.Series(np.random.randint(1,10, 10),index=dates)
print values
values.tail(4)

2013-01-25    1
2013-01-26    5
2013-01-27    2
2013-01-28    9
2013-01-29    5
2013-01-30    1
2013-01-31    4
2013-02-01    3
2013-02-02    4
2013-02-03    9
Freq: D, dtype: int64


2013-01-31    4
2013-02-01    3
2013-02-02    4
2013-02-03    9
Freq: D, dtype: int64

In [9]:

# We can select based on comparing the indexes
from datetime import datetime
end_january = datetime(2013, 01, 31)
february = values[values.index > end_january]
print february

print "-------"

# And also comparing the values
higher_than_five = values[values >5]
print higher_than_five

2013-02-01    3
2013-02-02    4
2013-02-03    9
Freq: D, dtype: int64
-------
2013-01-28    9
2013-02-03    9
dtype: int64


## Series of strings can also be manipulated

In [16]:
names = pd.Series(["John Smith", "Bob Geldof", "Ron Paul", "Frank Miller"])
names = names.str.split(expand=True)
names.columns = ["nombre", "apellido"]
names.sort_values(by=["apellido"])

,nombre,apellido
1,Bob,Geldof
3,Frank,Miller
2,Ron,Paul
0,John,Smith


## Series do stats too

And there are a number of statistical functions on series also.

In [17]:
s1 = pd.Series(np.random.randn(1000))

s2 = pd.Series(np.random.randn(1000))

print s1.corr(s2)
print s1.cov(s2)


-0.00621913555535
-0.00613921042075


In [18]:
s1 = pd.Series(np.random.randn(1000))

s2 = pd.Series(s1 + np.random.randn(1000)/10)

print s1.corr(s2)
print s1.cov(s2)

0.994834782607
0.951141921885


More on statistical functions: http://pandas.pydata.org/pandas-docs/dev/computation.html

In [19]:
# compute one-way ANOVA P value 
# from 3 samples (for two equiv. t-test)
from scipy import stats  

s1 = pd.Series(np.random.randn(10000))

s2 = pd.Series(np.random.gamma(0.42 ,0.5, 10000))

s3 = pd.Series(np.random.gamma(0.43 ,0.5, 10000))

f_val, p_val = stats.f_oneway(s1, s2, s3)        
print "One-way ANOVA P =", p_val  

f_val, p_val = stats.f_oneway(s2, s3)        
print "One-way ANOVA P =", p_val 

One-way ANOVA P = 9.92880795879e-184
One-way ANOVA P = 0.111955914436


<code>f_oneway</code> tests the null hypothesis that two or more groups have the same population mean. The hypothesis is rejected if any of these probabilities is less than or equal to a small, fixed but arbitrarily pre-defined threshold value $\alpha$, which is referred to as the level of significance.

However, it has some assumptions: http://docs.scipy.org/doc/scipy-0.17.0/reference/generated/scipy.stats.f_oneway.html   Do they hold in our case? 

In [20]:
from scipy.stats import ttest_1samp
t, p = ttest_1samp(s1, 0)
# If p is large, we can accept that the population has the mean indicated.
print p

0.0988251966909
